# 简历分析工作流--聚焦结构化输出
> 由于涉及隐私数据，qwen3模型不能返回，可换其它模型

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pocketflow import Node,Flow

load_dotenv()

True

In [5]:
import yaml

In [6]:
def call_llm(prompt):
    client = OpenAI(
        base_url= os.getenv("BASE_URL"),
        api_key=os.getenv("DASHSCOPE_API_KEY")
    )
    res = client.chat.completions.create(
        model = os.getenv("MODEL_NAME"),
        messages = [{
            "role": "user",
            "content": prompt
        }]
    )
    return res.choices[0].message.content

In [7]:

class ResumeParseNode(Node):
    # 获取简历数据
    def prep(self, shared):
        return shared['resume_text']
    # 解析简历并按需求的格式输出
    def exec(self, resume_text):
        prompt = f"""
        请从这份简历中提取以下信息，并将其格式化为 YAML：
        {resume_text}
        现在请按照以下格式输出：
        ```yaml
        name: 名字  
        email: xxx@xxx.com  
        experience:  
            - title: 职位名称1  
              company: 公司1
            - title: 职位名称2  
              company: 公司2
        skills:  
            - 技能1  
            - 技能2   
            - 技能3
        ```"""
        res = call_llm(prompt)
        yaml_str = res.split("```yaml")[1].split("```")[0].strip()
        structured_data = yaml.safe_load(yaml_str)
        return structured_data
    # 将结果处理后保存
    def post(self, shared, prep_res, exec_res):
        shared['structured_data'] = exec_res

        print("\n=== 简历结构化摘要数据 ===\n")
        print(yaml.dump(exec_res,allow_unicode=True,sort_keys=False))
        print("========================")


In [8]:
# 新建简历处理节点
resume_node = ResumeParseNode()
# 新建简历流程
resume_flow = Flow(start=resume_node)

In [11]:
def main():
    shared = {}
    # 读取简历文件
    with open('./data/resume1.txt', 'r', encoding='utf8') as file:
        resume_text = file.read()
    shared['resume_text'] = resume_text
    resume_flow.run(shared)

if __name__ == '__main__':
    main()


=== 简历结构化摘要数据 ===

name: 张伟
email: xxx@xxx.com
experience:
- title: 职位名称1
  company: 公司1
- title: 职位名称2
  company: 公司2
skills:
- 技能1
- 技能2
- 技能3

